In [1]:
import pandas as pd
from tqdm import tqdm
import string
import os

In [2]:
from DnD4py.lookup_5e import DnDSpell, Roll20Spell

In [3]:
df = pd.read_csv('all_spells.csv')

In [4]:
spells = {}
failed = {}

In [5]:
names = sorted([s.lower().replace('’', '').replace('/', '').replace('\'', '') for s in df['Name'].values])

In [7]:
for i, name in tqdm(enumerate(names), total=len(names)):
    if name in spells:
        continue
    try:
        spells[name] = DnDSpell(name).as_dungeonsheets_class()
    except Exception as e1:
        try:
            spells[name] = Roll20Spell(name).as_dungeonsheets_class()
        except Exception as e2:
            failed[name] = (e1, e2)
            print(f'Failed: {name}')

100%|██████████| 461/461 [06:02<00:00,  1.27it/s]


In [8]:
def save_spell(name, text):
    filename = f'dungeonsheets/spells/spells_{name[0].lower()}.py'
    if os.path.isfile(filename):
        with open(filename, 'r') as f:
            current_text = f.read().strip()
    else:
        with open(filename, 'a') as f:
            f.write('from .spells import Spell\n\n\n')
        current_text = ''
    if text.strip() in current_text:
        raise ValueError('Text already exists in the file')
    with open(filename, 'a') as f:
        f.write(text + '\n')

In [9]:
for c in string.ascii_lowercase:
    filename = f'dungeonsheets/spells/spells_{c}.py'
    if os.path.isfile(filename):
        os.remove(filename)

In [10]:
for name in tqdm(sorted(spells.keys())):
    save_spell(name, spells[name])

100%|██████████| 460/460 [00:00<00:00, 2537.91it/s]


In [2]:
import dungeonsheets

In [3]:
from dungeonsheets.spells import *

In [4]:
tests = {}
for k, v in dungeonsheets.spells.__dict__.items():
    if isinstance(v, type) and issubclass(v, dungeonsheets.spells.Spell):
        tests[k] = v()

In [5]:
tests.pop('Spell')

"Unknown spell"

In [6]:
test_char = dungeonsheets.Character()

In [7]:
test_char._spells = test_char._spells_prepared = list(tests.values())

In [9]:
create_spellbook_pdf(test_char, 'Entire_Spellbook')

In [10]:
test_char.wield_weapon('Spear +1')

In [11]:
test_char.weapons

["Spear+1"]